In [ ]:
# This notebook takes samples from a multivariate normal distribution 
# and fits the correct parameters via PDMM in a distributed fashion.
import autograd.numpy as np
from autograd.numpy.random import multivariate_normal, normal, randint
%matplotlib inline
from matplotlib import pyplot as plt
from MRF_Node import Node
from Neighbour import Neighbour 

In [ ]:
# This function returns a local objective function for a particular set of samples.
# The function is iteratively defined for flexibility
def f(x_i,x_j,var):
    x = zip(x_i,x_j)
    f = lambda a,b,c: 0
    for i,j in x:
        f = lambda a,b,c,f=f: (np.power(i-a,2) + (c/2)*(i-a)*(j-b))/(2*var) + f(a,b,c)
    return f

In [ ]:
#Initialise parameters
u = np.asarray([1,1])
E = np.asarray([[1,0.1],[0.1,1]])
n = 1000
n_sample = 10
x = multivariate_normal(u,E,n).T

In [ ]:
# Initialise PDMM Graph

N_nodes = 2;
N_dim = 3
msg_dim = 3

G = []
var = 1
sample_index = randint(n,size=n_sample)
x_s = x[:,sample_index]
d_T = 1e-15
p = 1e-5

#initialise nodes

obj = f(x_s[0],x_s[1],var)
G.append(Node(0,N_dim,obj,p,d_T))
obj = f(x_s[1],x_s[0],var)
G.append(Node(1,N_dim,obj,p,d_T))

#insert forward neighbours
A_forward = np.asarray([[0,1,0],[1,0,0],[0,0,1]])
A_backward = -np.eye(N_dim)
c_ij = np.zeros([N_dim,1])
P_ij = 0.1*np.eye(N_dim)

for i in np.arange(N_nodes-1):
    neighbour = Neighbour(G[i+1],i+1,A_forward,c_ij,P_ij,msg_dim)
    G[i].Neighbours.append(neighbour)

#insert backward neighbours 
for i in np.arange(N_nodes-1)+1:
    neighbour = Neighbour(G[i-1],i-1,A_backward,c_ij,P_ij,msg_dim)
    G[i].Neighbours.append(neighbour)

In [ ]:
#Train
for i in range(10):
    sample_index = randint(n,size=n_sample)
    x_s = x[:,sample_index]

    obj = f(x_s[0],x_s[1],var)
    G[1].f= obj
    obj = f(x_s[1],x_s[0],var)
    G[1].f= obj

    for node in G:
        node.update()
        
    for node in G:
        node.finalise()
    print(i)